google eearth engine

In [1]:
import ee

# Path to your service account JSON key
SERVICE_ACCOUNT = 'sincere-sun-404310-1223ed4b5b60.json'

# Initialize Earth Engine with the service account
credentials = ee.ServiceAccountCredentials(None, SERVICE_ACCOUNT)
ee.Initialize(credentials)

# Test Earth Engine
image = ee.Image('CGIAR/SRTM90_V4')
stats = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee.Geometry.Point([10.0, 36.0]),  # Example: Tunisia
    scale=90
)

print("Mean elevation (Tunisia):", stats.getInfo())


Mean elevation (Tunisia): {'elevation': 151}
